In [1]:
import pandas as pd
import numpy as np
from ortools.linear_solver import pywraplp

import warnings
warnings.filterwarnings('ignore')

## Load the data

In [2]:
df = pd.read_csv("data/all_players_fpl.csv")

In [3]:
df.head()

,id,code,element_type,now_cost,name,team,team_code,goals_scored,assists,own_goals,penalties_missed,total_points,points_per_game,team_name,position,updated_at
0,1,154561,1,5.5,David Raya Martín,1,3,0,0,0,0,142,3.7,Arsenal,GKP,2025-08-09 15:03:09
1,2,109745,1,4.5,Kepa Arrizabalaga Revuelta,1,3,0,0,0,0,106,3.4,Arsenal,GKP,2025-08-09 15:03:09
2,3,463748,1,4.0,Karl Hein,1,3,0,0,0,0,0,0.0,Arsenal,GKP,2025-08-09 15:03:09
3,4,551221,1,4.0,Tommy Setford,1,3,0,0,0,0,0,0.0,Arsenal,GKP,2025-08-09 15:03:09
4,5,226597,2,6.0,Gabriel dos Santos Magalhães,1,3,3,2,0,0,117,4.2,Arsenal,DEF,2025-08-09 15:03:09


In [4]:
ID_COL = "code"

In [5]:
players = df[ID_COL].tolist()

In [6]:
player_data = df.set_index(ID_COL).to_dict('index')

In [7]:
# Pre-process players by position and club for easier constraint building
pos_to_players = {pos: df[df['position'] == pos][ID_COL].tolist() for pos in ['GKP', 'DEF', 'MID', 'FWD']}
club_to_players = {club: df[df['team_name'] == club][ID_COL].tolist() for club in df['team_name'].unique()}

## Model Implementation

In [8]:
# MODEL PARAMETER
BUDGET = 100
SQUAD_SIZE = 15
STARTING_XI_SIZE = 11
MAX_PLAYER_PER_TEAM = 3

POSITION_QUOTAS = {
    "GKP": 2,
    "DEF": 5,
    "MID": 5,
    "FWD": 3
}

STARTING_POSITION_LIMITS = {
    "GKP": (1,1),
    "DEF": (3, 5),
    "MID": (3, 5),
    "FWD": (1, 3)
}

In [9]:
OBJECTIVE_VAR = "points_per_game"

In [10]:
solver = pywraplp.Solver.CreateSolver('SCIP')


# Decision variables
squad_vars = {p: solver.BoolVar(f'squad_{p}') for p in players}
starts_vars = {p: solver.BoolVar(f'starts_{p}') for p in players}
captain_vars = {p: solver.BoolVar(f'captain_{p}') for p in players}


# 1. Define Constraint
# A. Squad Selection Constraints
# Budget constraint
total_cost = solver.Sum([player_data[p]['now_cost'] * squad_vars[p] for p in players])
solver.Add(total_cost <= BUDGET)

# Squad size constraint
solver.Add(solver.Sum(squad_vars.values()) == SQUAD_SIZE)

# Squad position constraints
for pos, count in POSITION_QUOTAS.items():
    solver.Add(solver.Sum([squad_vars[p] for p in pos_to_players[pos]]) == count)

# Max players per team constraint
for club in club_to_players:
    solver.Add(solver.Sum([squad_vars[p] for p in club_to_players[club]]) <= MAX_PLAYER_PER_TEAM)

# B. Starting XI Formation Constraints
# Starting XI size
solver.Add(solver.Sum(starts_vars.values()) == STARTING_XI_SIZE)

# Starting XI position constraints
for pos, (min_val, max_val) in STARTING_POSITION_LIMITS.items():
    solver.Add(solver.Sum([starts_vars[p] for p in pos_to_players[pos]]) >= min_val)
    solver.Add(solver.Sum([starts_vars[p] for p in pos_to_players[pos]]) <= max_val)

# C. Captaincy Constraints
# Exactly one captain
solver.Add(solver.Sum(captain_vars.values()) == 1)

# D. Logical Linking Constraints
for p in players:
    # A player must be in the squad to start
    solver.Add(starts_vars[p] <= squad_vars[p])
    # A player must be a starter to be captain
    solver.Add(captain_vars[p] <= starts_vars[p])

# --- 5. Define the Objective Function ---
# Maximize total points from starters + captain bonus
starters_points = solver.Sum([player_data[p][OBJECTIVE_VAR] * starts_vars[p] for p in players])
captain_points = solver.Sum([player_data[p][OBJECTIVE_VAR] * captain_vars[p] for p in players])

solver.Maximize(starters_points + captain_points)

# --- 6. Solve the Model ---
print("Solving the FPL optimization model...")
status = solver.Solve()

# --- 7. Retrieve the Results ---
if status == pywraplp.Solver.OPTIMAL:
    print("\n--- Optimal FPL Squad Found! ---\n")
    
    selected_captain = ""
    selected_starters = []
    selected_squads = []
    total_squad_cost = 0
    
    for p in players:
        if captain_vars[p].solution_value() == 1:
            selected_captain = p
        if starts_vars[p].solution_value() == 1:
            selected_starters.append(p)
        if squad_vars[p].solution_value() == 1:
            selected_squads.append(p)

        if squad_vars[p].solution_value() == 1:
            total_squad_cost += player_data[p]['now_cost']
    
    print(f"Optimal Solution: {solver.Objective().Value():.2f}")
    print(f"Total Squad Cost: £{total_squad_cost:.1f}m\n")

Solving the FPL optimization model...

--- Optimal FPL Squad Found! ---

Optimal Solution: 68.60
Total Squad Cost: £100.0m



In [11]:
result_df = df[df[ID_COL].isin(selected_squads)]

In [12]:
result_df.loc[:,"is_captain"] = result_df[ID_COL] == selected_captain

In [13]:
result_df.loc[:, "is_starter"] = result_df[ID_COL].isin(selected_starters)

In [14]:
result_df = result_df.sort_values(["is_starter", "element_type"], ascending=[False, True])

In [15]:
filter_cols = ["name", "position", "team_name", "is_starter", "is_captain", "now_cost", "points_per_game"]

In [16]:
final_df = result_df[filter_cols]

## Starting XI

In [17]:
final_df[final_df["is_starter"]==True]

,name,position,team_name,is_starter,is_captain,now_cost,points_per_game
646,Daniel Bentley,GKP,Wolves,True,False,4.0,4.5
4,Gabriel dos Santos Magalhães,DEF,Arsenal,True,False,6.0,4.2
228,Trevoh Chalobah,DEF,Chelsea,True,False,5.0,3.7
514,Nikola Milenković,DEF,Nott'm Forest,True,False,5.5,3.9
388,Mohamed Salah,MID,Liverpool,True,True,14.5,9.1
440,Bryan Mbeumo,MID,Man Utd,True,False,8.0,6.2
460,Matheus Santos Carneiro da Cunha,MID,Man Utd,True,False,8.0,5.4
462,Amad Diallo,MID,Man Utd,True,False,6.5,5.0
507,Alexander Isak,FWD,Newcastle,True,False,10.5,6.2
534,Chris Wood,FWD,Nott'm Forest,True,False,7.5,5.6


## Bench

In [18]:
final_df[final_df["is_starter"]==False]

,name,position,team_name,is_starter,is_captain,now_cost,points_per_game
145,Julian Eyestone,GKP,Brentford,False,False,4.0,0.0
656,Bastien Meupiyou Menadjou,DEF,Wolves,False,False,4.0,0.0
657,Pedro Cardoso de Lima,DEF,Wolves,False,False,4.0,0.7
635,George Earthy,MID,West Ham,False,False,4.5,0.0
